In [270]:
import pandas as pd
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Dict, Iterable, List, Tuple
import csv
import io
import math

In [271]:
# ---------- Utilitaires ----------

def fr_float(s: str | None) -> Optional[float]:
    """Convertit une chaîne avec virgule en float (ou None si vide)."""
    if s is None:
        return None
    s = s.strip().strip('"').strip("'")
    if s == "" or s.lower() in {"nan", "null"}:
        return None
    # Remplacer la virgule décimale par un point
    return float(s.replace(",", "."))


Définition du mur

In [272]:
# ---------- Modèle de données ----------
@dataclass
class Mur:
    """Représente un mur avec ses coordonnées/mesures."""
    id: str
    x: Optional[float]
    y: Optional[float]
    z: Optional[float]
    centre_x: Optional[float]
    centre_y: Optional[float]
    centre_z: Optional[float]
    niv_sol: Optional[float]

    @staticmethod
    def from_csv_row(row: dict) -> "Mur":
        """Crée un Mur depuis une ligne CSV (clés: Murs, X, Y, Z)."""
        return Mur(
            id=row.get("Murs", "").strip(),
            x=fr_float(row.get("X")),
            y=fr_float(row.get("Y")),
            z=fr_float(row.get("Z")),
            centre_x=fr_float(row.get("centre_x")),
            centre_y=fr_float(row.get("centre_y")),
            centre_z=fr_float(row.get("centre_z")),
            niv_sol=fr_float(row.get("niv_sol")),
        )

Chargement des murs et frontieres

In [273]:
# ---------- Parsing du CSV ----------

def lire_murs_csv(source: str | io.TextIOBase) -> Dict[str, Mur]:
    """
    Lit un CSV et renvoie un dict {id_mur: Mur}.
    Accepte des valeurs X/Y/Z avec virgule comme décimale.
    """
    stream = open(source, "r", encoding="utf-8")

    reader = csv.DictReader(stream)
    murs: Dict[str, Mur] = {}
    for row in reader:
        mur = Mur.from_csv_row(row)
        if not mur.id:
            # Ignore les lignes sans identifiant
            continue
        murs[mur.id] = mur
    return murs

In [274]:
# ---------- Exemple d’utilisation ----------

murs = lire_murs_csv("./data/murs.csv")

print(murs)                       


{'0AX0': Mur(id='0AX0', x=6.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0AX1': Mur(id='0AX1', x=6.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0AY0': Mur(id='0AY0', x=0.15, y=5.6, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0AY1': Mur(id='0AY1', x=0.15, y=5.6, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0BX0': Mur(id='0BX0', x=6.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0BX1': Mur(id='0BX1', x=6.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0BY0': Mur(id='0BY0', x=0.15, y=3.0, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0BY1': Mur(id='0BY1', x=0.15, y=3.0, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0CX0': Mur(id='0CX0', x=6.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None, niv_sol=0.0), '0CX1': Mur(id='0CX1', x=6.0, y=0.15, z=3.0, centre_x=None, cen

In [275]:
def lire_frontieres_csv(source: str | io.TextIOBase) -> List[Tuple[str, str]]:
    """
    Lit un CSV de frontières (colonnes: Mur1, Mur2)
    et renvoie une liste de couples (mur1, mur2) uniques.
    """
    stream = open(source, "r", encoding="utf-8")
    reader = csv.DictReader(stream)
    frontieres_set = set()  # pour éviter les doublons

    for row in reader:
        m1 = row.get("Mur1", "").strip()
        m2 = row.get("Mur2", "").strip()
        if not m1 or not m2:
            continue

        # Tri pour éviter les inversions (("A","B") == ("B","A"))
        pair = tuple(sorted((m1, m2)))
        frontieres_set.add(pair)

    # Ferme le fichier si on l’a ouvert ici
    if isinstance(source, (str, bytes)):
        stream.close()

    # Convertit l’ensemble en liste triée
    return frontieres_set


Trouver la longueur et largeur

In [276]:
def frontiere_nord(piece, frontieres):
    target = "0"+piece+"X1"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    return False 

In [277]:
def longueur_totale_y(murs, frontieres):
    l_tot = 0
    piece_select = "DEPART"

    while piece_select != False :
        piece_select = frontiere_nord(piece_select, frontieres)
        if piece_select !=False :
            l_tot += ( murs["0" + piece_select + "X0"].y ) + murs["0" + piece_select + "Y0"].y + ( murs["0" + piece_select + "X1"].y )   #Le mur + l'épaisseur des murs en travers
    return round(l_tot, 3)

In [278]:
def frontiere_ouest(piece, frontieres):
    target = "0"+piece+"Y0"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    return False 

In [279]:
def largeur_totale_x(murs, frontieres):
    l_tot = 0
    piece_select = "DEPART"

    while piece_select != False :
        piece_select = frontiere_ouest(piece_select, frontieres)
        if piece_select !=False :
            l_tot += ( murs["0" + piece_select + "Y0"].x ) + murs["0" + piece_select + "X0"].x + ( murs["0" + piece_select + "Y1"].x )   #Le mur + l'épaisseur des murs en travers
    return round(l_tot, 3) 

In [280]:
murs = lire_murs_csv("./data/murs.csv")
frontieres = lire_frontieres_csv("./data/frontiere.csv")

print(longueur_totale_y(murs,frontieres))      
print(largeur_totale_x(murs,frontieres))              

21.8
6.3


Trouver les positions

In [281]:
def frontiere_est(piece, frontieres, num_etage):
    target = str(num_etage) + piece+"Y1"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    raise ValueError(f"Pas trouvé de frontière est pour {target}")

In [282]:
def frontiere_sud(piece, frontieres, num_etage):
    target = str(num_etage)+piece+"X0"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    raise ValueError(f"Pas trouvé de frontière sud pour {target}")

In [ ]:
def place_piece(piece, murs, frontieres, largeur_totale_x, longueur_totale_y, num_etage) :
    fsud = frontiere_sud(piece,frontieres, num_etage)
    fest = frontiere_est(piece,frontieres, num_etage)

    max_x = largeur_totale_x/2
    max_y = longueur_totale_y/2

    #Mur du bas X0 : 

    mur = murs[str(num_etage) + piece + "X0"]
    #centre_y
    if(fsud) == "DEPART" :
        centre_y = -1 * max_y + mur.y /2
        mur.centre_y = round(centre_y, 3) 
    else :
        centre_y = murs[str(num_etage) + fsud + "X1"].centre_y + murs[str(num_etage) + fsud + "X1"].y /2 + mur.y /2
                    #  La position du mur de frontiere + la moitié du mur de frontiere + la moitié du nouveau mur (son centre)
        mur.centre_y = round(centre_y, 3) 
    
    #centre_x
    if(fest) == "DEPART" :
        centre_x = max_x - murs[str(num_etage) + piece + "Y1"].x - mur.x /2
        mur.centre_x = round(centre_x, 3) 
    else :
        centre_x = murs[str(num_etage) + fest + "Y0"].centre_x - murs[str(num_etage) + fest + "Y0"].x / 2 - murs[str(num_etage) + piece + "Y1"].x - mur.x/2
        mur.centre_x = round(centre_x, 3) 

    #centre_z
    centre_z = mur.niv_sol + mur.z/2
    mur.centre_z = centre_z

    #Mur du haut X1 : 

    mur = murs[str(num_etage) + piece + "X1"]
    #centre_y
    centre_y =  murs[str(num_etage) + piece + "X0"].centre_y + murs[str(num_etage) + piece + "X0"].y / 2 + murs[str(num_etage) + piece + "Y0"].y + mur.y / 2
    mur.centre_y = round(centre_y, 3) 

    #centre_x
    centre_x = murs[str(num_etage) + piece + "X0"].centre_x
    mur.centre_x = round(centre_x, 3) 

    #centre_z
    centre_z = mur.niv_sol + mur.z/2
    mur.centre_z = centre_z

    #Mur de droite Y1 :
    mur = murs[str(num_etage) + piece + "Y1"]
    #centre_x
    if(fest) == "DEPART" :
        centre_x = max_x - mur.x / 2
        mur.centre_x = round(centre_x, 3) 
    else :
        centre_x = murs[str(num_etage) + fest + "Y0"].centre_x - murs[str(num_etage) + fest + "Y0"].x /2 - mur.x / 2
        mur.centre_x = round(centre_x, 3) 

    #centre_y
    if(fsud) == "DEPART" :
        centre_y = -1 * max_y + murs[str(num_etage) + piece + "X0"].y + mur.y/2
        mur.centre_y = round(centre_y, 3) 
    else :
        centre_y = murs[str(num_etage) + piece + "X0"].centre_y + murs[str(num_etage) + piece + "X0"].y /2 + mur.y/2
        mur.centre_y = round(centre_y, 3) 

    #centre_z
    centre_z = mur.niv_sol + mur.z/2
    mur.centre_z = centre_z

    #Mur de Gauche Y0 :

    mur = murs[str(num_etage) + piece + "Y0"]
    #centre_x
    centre_x = murs[str(num_etage) + piece + "X0"].centre_x - murs[str(num_etage) + piece + "X0"].x / 2 - mur.x /2
    mur.centre_x = round(centre_x, 3) 

    #centre_y
    centre_y = murs[str(num_etage) + piece + "Y1"].centre_y
    mur.centre_y = round(centre_y, 3) 

    #centre_z
    centre_z = mur.niv_sol + mur.z/2
    mur.centre_z = centre_z



In [284]:
liste_des_pieces = [["A","B","C","D","E","F","G"], ["AB","AA","AC","B","C","D","E","FA","FB"], ["AB","AA","AC","B","C","D"]]

murs = lire_murs_csv("./data/murs.csv")
frontieres = lire_frontieres_csv("./data/frontiere.csv")

long_tot_y = longueur_totale_y(murs,frontieres)
larg_tot=x = largeur_totale_x(murs,frontieres)

num_etage = 0

for etage in liste_des_pieces :
    for piece in etage :
        place_piece(piece, murs, frontieres, larg_tot, long_tot_y, num_etage)
    num_etage += 1

0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.5
3.5
3.5
3.5
3.5
3.5
3.5
3.5
3.5
7.0
7.0
7.0
7.0
7.0
7.0


In [285]:
import csv
from typing import Dict
from dataclasses import asdict

def ecrire_murs_csv(murs: Dict[str, Mur], chemin: str) -> None:
    """
    Exporte le dictionnaire `murs` dans un fichier CSV.
    Colonnes : Murs, X, Y, Z, centre_x, centre_y, centre_z
    Les floats sont formatés avec virgule décimale.
    """
    champs = ["Murs", "X", "Y", "Z", "centre_x", "centre_y", "centre_z"]

    with open(chemin, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=champs)
        writer.writeheader()

        for mur in murs.values():
            # Conversion en dict de base
            d = asdict(mur)
            # Format des nombres avec virgule décimale (ou vide si None)
            def fmt(v):
                if v is None:
                    return ""
                if isinstance(v, (float, int)):
                    return str(v).replace(".", ",")
                return str(v)

            ligne = {
                "Murs": mur.id,
                "X": fmt(mur.x),
                "Y": fmt(mur.y),
                "Z": fmt(mur.z),
                "centre_x": fmt(mur.centre_x),
                "centre_y": fmt(mur.centre_y),
                "centre_z": fmt(mur.centre_z),
            }
            writer.writerow(ligne)

    print(f"✅ Fichier CSV écrit : {chemin}")


In [286]:
# Suppose que murs = {"0AX0": Mur(...), "0AX1": Mur(...), ...}

ecrire_murs_csv(murs, "./data/murs_export.csv")


✅ Fichier CSV écrit : ./data/murs_export.csv
